In [104]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import time
import random
pd.set_option("display.max_columns", None)

In [9]:
sample_df = pd.read_csv('sample_superstore.csv')
sample_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [11]:
import findspark
findspark.init()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession, Row

spark = SparkSession\
    .builder\
    .appName("furniture_analytics")\
    .getOrCreate()

In [38]:
# Set options for reading the data.
options = {
"header": True,
"inferSchema": True,
}

sample_df = spark.read\
            .options(**options).csv('sample_superstore.csv')
            
sample_df.limit(5).toPandas()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164


In [78]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   row_id         9994 non-null   int64  
 1   order_id       9994 non-null   object 
 2   order_date     9994 non-null   object 
 3   ship_date      9994 non-null   object 
 4   ship_mode      9994 non-null   object 
 5   customer_id    9994 non-null   object 
 6   customer_name  9994 non-null   object 
 7   segment        9994 non-null   object 
 8   country        9994 non-null   object 
 9   city           9994 non-null   object 
 10  state          9994 non-null   object 
 11  postal_code    9994 non-null   int64  
 12  region         9994 non-null   object 
 13  product_id     9994 non-null   object 
 14  category       9994 non-null   object 
 15  sub_category   9994 non-null   object 
 16  product_name   9994 non-null   object 
 17  sales          9994 non-null   float64
 18  quantity

In [41]:
sample_df\
    .groupBy('customer_id')\
    .agg(countDistinct(col('customer_name')).alias('num_name'), countDistinct(col('country')).alias('num_country'))\
    .orderBy(col('num_name').desc())\
    .limit(5).toPandas()

,customer_id,num_name,num_country
0,RR-19315,1,1
1,PB-19210,1,1
2,VW-21775,1,1
3,MS-17530,1,1
4,EM-13960,1,1


In [42]:
sample_df = pd.read_csv('sample_superstore.csv')
sample_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [55]:
def extract_customer_info(sample_df):
    customer_info_df = sample_df.loc[:, ["product_id", "customer_name", 'segment', 'country', 'city']]
    customer_info_df = customer_info_df.drop_duplicates()
    return customer_info_df

In [56]:
customer_info_df = extract_customer_info(sample_df)

In [57]:
customer_info_df.head()

,customer_id,customer_name,segment,country,city
0,CG-12520,Claire Gute,Consumer,United States,Henderson
2,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles
3,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale
5,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles
12,AA-10480,Andrew Allen,Consumer,United States,Concord


In [58]:
customer_info_df.to_dict('records')[4]

{'customer_id': 'AA-10480',
 'customer_name': 'Andrew Allen',
 'segment': 'Consumer',
 'country': 'United States',
 'city': 'Concord'}

In [59]:
def generate_customer_info(customer_info_df):
    n = np.random.randint(0, len(customer_info_df))
    return customer_info_df.to_dict('records')[n]

In [83]:
def calculate_price(row):
    return row['sales'] / row['quantity'] / (1 - row['discount'])

def extract_product_info(sample_df):
    product_info = sample_df.loc[:, ["category", 'sub_category', 'product_name', 'sales', 'quantity', 'discount']]
    product_info['price'] = product_info.apply(calculate_price, axis=1)
    product_info = product_info.loc[:, ["category", 'sub_category', 'product_name', 'price']]
    product_info = product_info.drop_duplicates()
    return product_info

In [84]:
product_info = extract_product_info(sample_df)
product_info.head()

,category,sub_category,product_name,price
0,Furniture,Bookcases,Bush Somerset Collection Bookcase,130.98
1,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",243.98
2,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,7.31
3,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,348.21
4,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,13.98


In [85]:
def generate_product_info(product_info):
    n = np.random.randint(0, len(product_info))
    return product_info.to_dict('records')[n]

In [96]:
generate_product_info(product_info)

{'category': 'Office Supplies',
 'sub_category': 'Storage',
 'product_name': 'Portfile Personal File Boxes',
 'price': 17.699999999999996}

In [109]:
def generate_date_time():
    day = random.randint(1, 31)
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)

    timestamp = datetime(2023, 12, day, hour, minute, second)

    return timestamp.strftime('%Y-%m-%d %H:%M:%S')

In [110]:
generate_date_time()

'2023-12-17 05:49:07'

In [137]:
def generate_log(n_records):
    customer_info_df = extract_customer_info(sample_df)
    product_info_df = extract_product_info(sample_df)
    discount_list = [0.5, 0.4, 0.3, 0]
    
    for _ in range(n_records):
        customer_info_dict = generate_customer_info(customer_info_df)
        product_info_dict = generate_product_info(product_info_df)
        quantity = random.randint(1,20)
        discount = random.choices(discount_list, weights=(0.5,0.15,0.2,0.6))[0]
        ts = generate_date_time()
        
        data = {'ts':f'{ts}',
                'customer_id':customer_info_dict['customer_id'],
                'customer_name':customer_info_dict['customer_name'],
                'segment':customer_info_dict['segment'],
                'country':customer_info_dict['country'],
                'city':customer_info_dict['city'],
                'category':product_info_dict['category'],
                'sub_category':product_info_dict['sub_category'],
                'product_name':product_info_dict['product_name'],
                'price':product_info_dict['price'],
                'quantity':f'{quantity}',
                'discount':f'{discount}',
                'revenue': float(product_info_dict['price'])*quantity*(1-discount)
                }
        print(f'Sending data: {data}')

In [138]:
generate_log(n_records=4)

Sending data: {'ts': '2023-12-08 01:49:00', 'customer_id': 'SF-20965', 'customer_name': 'Sylvia Foulston', 'segment': 'Corporate', 'country': 'United States', 'city': 'Baytown', 'category': 'Furniture', 'sub_category': 'Bookcases', 'product_name': 'Bush Westfield Collection Bookcases, Fully Assembled', 'price': 100.98000000000002, 'quantity': '5', 'discount': '0', 'revenue': 504.9000000000001}
Sending data: {'ts': '2023-12-11 11:45:31', 'customer_id': 'MF-18250', 'customer_name': 'Monica Federle', 'segment': 'Corporate', 'country': 'United States', 'city': 'New York City', 'category': 'Furniture', 'sub_category': 'Furnishings', 'product_name': 'Floodlight Indoor Halogen Bulbs, 1 Bulb per Pack, 60 Watts', 'price': 19.400000000000002, 'quantity': '18', 'discount': '0.4', 'revenue': 209.52}
Sending data: {'ts': '2023-12-31 23:57:36', 'customer_id': 'KN-16705', 'customer_name': 'Kristina Nunn', 'segment': 'Home Office', 'country': 'United States', 'city': 'Mcallen', 'category': 'Furniture'

In [114]:
ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
ts

'2023-12-13 00:11:56'

In [111]:
customer_info_df.columns

Index(['customer_id', 'customer_name', 'segment', 'country', 'city'], dtype='object')

In [127]:
discount_list = ['0.5', '0.4', '0.3', '0']
random.choices(discount_list, weights=(5,15,20,60))[0]

'0.4'

In [ ]:
        quantity = random.randint(1,20)
        discount = random.choices(discount_list, weights=(0.5,0.15,0.2,0.6))[0]


In [135]:
product_info_df = extract_product_info(sample_df)
product_info_dict = generate_product_info(product_info_df)
float(product_info_dict['price'])

130.98

In [136]:
quantity = random.randint(1,20)
quantity

1